In [41]:
from datetime import datetime
import numpy as np
import torch
import argparse
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append("../../")
from TCN.soil_classification.model import TCN
from TCN.soil_classification.utils import data_generator
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix
from sklearn import metrics

### Value Indexing

In [2]:
LIN_ACC_X = 1
LIN_ACC_Y = 2
LIN_ACC_Z = 3
ANG_VEL_X = 4
ANG_VEL_Y = 5
ANG_VEL_Z = 6
ORIENT_X = 7
ORIENT_Y = 8
ORIENT_Z = 9
ORIENT_W = 10
POS_X = 11
POS_Z = 12
ANG = 13
BOOM = 14
DIPPER = 15
TELE = 16
PITCH = 17

In [3]:
seed = 1111
folder = "/home/mads/git/TCN/TCN/soil_classification/data/prelim"
no_air = True

val_mask = range(LIN_ACC_X,ANG_VEL_Z+1,1)

X, Y = data_generator(folder, 100, 50, val_mask=val_mask)

shuffle_seq = np.arange(X.shape[0])
rng = np.random.default_rng(seed)
rng.shuffle(shuffle_seq)
X = X[shuffle_seq,:,:]
Y = Y[shuffle_seq]

X,Y = Variable(torch.from_numpy(X)), Variable(torch.from_numpy(Y))

if no_air:
    remove_air = Y>0
    X, Y = X[remove_air], Y[remove_air]

input_channels = len(val_mask)
n_classes = 3
batch_size = 32
seq_length = 500
epochs = 10
nhid = 30
levels = 8
ksize = 7

channel_sizes = [nhid]*levels
kernel_size = ksize
dropout = 0.0
model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=dropout)

lr = 4e-3
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)

test_pct = 15

X_train, X_test = X[int(X.size()[0]/100*test_pct):], X[:int(X.size()[0]/100*test_pct)]
Y_train, Y_test = Y[int(X.size()[0]/100*test_pct):], Y[:int(X.size()[0]/100*test_pct)]

In [4]:
mins, _ = torch.min(torch.min(X_train,1)[0],0)
maxs, _ = torch.max(torch.max(X_train,1)[0],0)
X_train, X_test = (X_train-mins)/(maxs-mins), (X_test-mins)/(maxs-mins)
X_train, X_test = X_train.permute(0,2,1), X_test.permute(0,2,1)

In [5]:
X_train.size()

torch.Size([806, 6, 100])

In [6]:
def train(epoch):
    global lr
    model.train()
    batch_idx = 1
    total_loss = 0
    full_loss = 0
    idx = np.arange(X_train.size(0))
    for i in range(0, X_train.size(0), batch_size):
        rng.shuffle(idx)
        if i + batch_size > X_train.size(0):
            x, y = X_train[idx[i:]], Y_train[idx[i:]]
        else:
            x, y = X_train[idx[i:(i+batch_size)]], Y_train[idx[i:(i+batch_size)]]
        optimizer.zero_grad()
        output = model(x)
        loss = F.nll_loss(output, y)
        loss.backward()
        if clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()
        batch_idx += 1
        total_loss += loss.item()
        full_loss += loss.item()

        if batch_idx % log_interval == 0:
            cur_loss = total_loss / log_interval
            processed = min(i+batch_size, X_train.size(0))
            print('Train Epoch: {:2d} [{:6d}/{:6d} ({:.0f}%)]\tLearning rate: {:.4f}\tLoss: {:.6f}'.format(
                epoch, processed, X_train.size(0), 100.*processed/X_train.size(0), lr, cur_loss))
            total_loss = 0
    full_loss = full_loss/batch_idx
    return full_loss
            
def evaluate():
    model.eval()
    with torch.no_grad():
        output = model(X_test)
        test_loss = F.nll_loss(output, Y_test)
        print('\nTest set: Average loss: {:.6f}\n'.format(test_loss.item()))
        return test_loss.item()

In [7]:
clip  = -1
log_interval = 25
train_loss = []
test_loss = []
for ep in range(1, epochs+1):
    train_loss.append(train(ep))
    test_loss.append(evaluate())
FORMAT = '%Y-%m-%-d_%H%M%S'
stamp = datetime.now().strftime(FORMAT)
with open('log/train_test_'+stamp+'.csv', 'w') as f:
    f.write(f"epoch, train_loss, test_loss\n")
    for ep,(train, test) in enumerate(zip(train_loss, test_loss)):
        f.write(f"{ep},{train}, {test}\n")

Train Epoch:  1 [   768/   806 (95%)]	Learning rate: 0.0040	Loss: 0.696322

Test set: Average loss: 0.655909

Train Epoch:  2 [   768/   806 (95%)]	Learning rate: 0.0040	Loss: 0.460840

Test set: Average loss: 0.356722

Train Epoch:  3 [   768/   806 (95%)]	Learning rate: 0.0040	Loss: 0.203853

Test set: Average loss: 0.163541

Train Epoch:  4 [   768/   806 (95%)]	Learning rate: 0.0040	Loss: 0.054207

Test set: Average loss: 0.105159

Train Epoch:  5 [   768/   806 (95%)]	Learning rate: 0.0040	Loss: 0.050365

Test set: Average loss: 0.045028

Train Epoch:  6 [   768/   806 (95%)]	Learning rate: 0.0040	Loss: 0.025937

Test set: Average loss: 0.056419

Train Epoch:  7 [   768/   806 (95%)]	Learning rate: 0.0040	Loss: 0.087632

Test set: Average loss: 0.058140

Train Epoch:  8 [   768/   806 (95%)]	Learning rate: 0.0040	Loss: 0.026338

Test set: Average loss: 0.057410

Train Epoch:  9 [   768/   806 (95%)]	Learning rate: 0.0040	Loss: 0.099749

Test set: Average loss: 0.046719

Train Epoc

In [ ]:
N=100
X_num = torch.rand([N, 1, seq_length])
X_mask = torch.zeros([N, 1, seq_length])
Y = torch.zeros([N, 1])
for i in range(N):
    positions = np.random.choice(seq_length, size=2, replace=False)
    X_mask[i, 0, positions[0]] = 1
    X_mask[i, 0, positions[1]] = 1
    Y[i,0] = X_num[i, 0, positions[0]] + X_num[i, 0, positions[1]]
X = torch.cat((X_num, X_mask), dim=1)
add_X, add_Y = Variable(X), Variable(Y)

In [ ]:
add_X.type()

In [ ]:
model.eval()
with torch.no_grad():
    output = model(X_train)

In [ ]:
pred = np.argmax(output.numpy(),axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, pred)

In [8]:
model.eval()
with torch.no_grad():
    output = model(X_train)
pred = np.argmax(output.numpy(),axis=1)
confusion_matrix(Y_train, pred)

array([[368,   4],
       [  0, 434]])

In [9]:
model.eval()
with torch.no_grad():
    output = model(X_test)
pred = np.argmax(output.numpy(),axis=1)
confusion_matrix(Y_test, pred)

array([[64,  3],
       [ 0, 75]])

In [43]:
report = metrics.classification_report(Y_test, pred, output_dict = True)
print(report)

{'1': {'precision': 1.0, 'recall': 0.9552238805970149, 'f1-score': 0.9770992366412213, 'support': 67}, '2': {'precision': 0.9615384615384616, 'recall': 1.0, 'f1-score': 0.9803921568627451, 'support': 75}, 'accuracy': 0.9788732394366197, 'macro avg': {'precision': 0.9807692307692308, 'recall': 0.9776119402985075, 'f1-score': 0.9787456967519832, 'support': 142}, 'weighted avg': {'precision': 0.9796858071505958, 'recall': 0.9788732394366197, 'f1-score': 0.9788384550680824, 'support': 142}}


In [10]:
X_test

tensor([[[0.6322, 0.6322, 0.6322,  ..., 0.6364, 0.6364, 0.6405],
         [0.3216, 0.3216, 0.3222,  ..., 0.3271, 0.3271, 0.3271],
         [0.9244, 0.9239, 0.9239,  ..., 0.9371, 0.9367, 0.9367],
         [0.3011, 0.3011, 0.3008,  ..., 0.3253, 0.3253, 0.3251],
         [0.4425, 0.4425, 0.4454,  ..., 0.4483, 0.4483, 0.4536],
         [0.5868, 0.5868, 0.5900,  ..., 0.5835, 0.5835, 0.5892]],

        [[0.6983, 0.6901, 0.6860,  ..., 0.7893, 0.7851, 0.7893],
         [0.4040, 0.4047, 0.4047,  ..., 0.3645, 0.3631, 0.3631],
         [0.5614, 0.5619, 0.5629,  ..., 0.5836, 0.5841, 0.5841],
         [0.5990, 0.5992, 0.5996,  ..., 0.5754, 0.5754, 0.5745],
         [0.4991, 0.4980, 0.4974,  ..., 0.5108, 0.5108, 0.5096],
         [0.6247, 0.6239, 0.6239,  ..., 0.6941, 0.6941, 0.6933]],

        [[0.4380, 0.4380, 0.4339,  ..., 0.4669, 0.4587, 0.4545],
         [0.7477, 0.7457, 0.7464,  ..., 0.7346, 0.7387, 0.7422],
         [0.4022, 0.4060, 0.4083,  ..., 0.3634, 0.3634, 0.3625],
         [0.2144, 0.2

In [33]:
Y_train

tensor([1, 1, 2, 1, 2, 1, 2, 2, 2, 0, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1,
        2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 0, 1, 0, 2, 2, 0, 2, 1, 2, 2, 1,
        2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 0, 2, 2, 0, 1, 2, 2, 2,
        2, 1, 2, 2, 1, 2, 0, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 1, 2, 2, 1, 0, 0, 1,
        1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 0, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1,
        1, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1,
        2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2,
        2, 2, 1, 1, 0, 2, 2, 1, 2, 2, 0, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2,
        1, 0, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 0, 1, 2,
        2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 0, 2, 1, 0, 2, 2,
        2, 1, 1, 1, 2, 0, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1,
        2, 2, 2, 0, 1, 1, 2, 2, 2, 2, 1,